# RL with and without function approximation


In [ ]:
#from IPython import get_ipython
import rlberry.colab_utils.display_setup
from rlberry.colab_utils.display_setup import show_video
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
%%javascript
(function(on) {
const e=$( "<a>Setup failed</a>" );
const ns="js_jupyter_suppress_warnings";
var cssrules=$("#"+ns);
if(!cssrules.length) cssrules = $("<style id='"+ns+"' type='text/css'>div.output_stderr { } </style>").appendTo("head");
e.click(function() {
    var s='Showing';  
    cssrules.empty()
    if(on) {
        s='Hiding';
        cssrules.append("div.output_stderr, div[data-mime-type*='.stderr'] { display:none; }");
    }
    e.text(s+' warnings (click to toggle)');
    on=!on;
}).click();
$(element).append(e);
})(true);

# Preparation

In the coding exercises, you will use a *grid-world* MDP, which is represented in Python using the interface provided by the [Gym](https://gym.openai.com/) library. The cells below show how to interact with this MDP and how to visualize it.


In [ ]:
from rlberry.envs import GridWorld

def get_env():
  """Creates an instance of a grid-world MDP."""
  env = GridWorld(
      nrows=5,
      ncols=7,
      reward_at = {(0, 6):1.0},
      walls=((0, 4), (1, 4), (2, 4), (3, 4)),
      success_probability=0.9,
      terminal_states=((0, 6),)
  )
  return env

def render_policy(env, policy=None, horizon=50):
  """Visualize a policy in an environment

  Args:
    env: GridWorld
        environment where to run the policy
    policy: np.array
        matrix mapping states to action (Ns).
        If None, runs random policy.
    horizon: int
        maximum number of timesteps in the environment.
  """
  env.enable_rendering()
  state = env.reset()                       # get initial state
  for timestep in range(horizon):
      if policy is None:
        action = env.action_space.sample()  # take random actions
      else:
        action = policy[state]
      next_state, reward, is_terminal, info = env.step(action)
      state = next_state
      if is_terminal:
        break
  # save video and clear buffer
  env.save_video('./videos/gw.mp4', framerate=5)
  env.clear_render_buffer()
  env.disable_rendering()
  # show video
  show_video('./videos/gw.mp4')


In [ ]:
# Create an environment and visualize it
env = get_env()
render_policy(env)  # visualize random policy

# The reward function and transition probabilities can be accessed through
# the R and P attributes:
print(f"Shape of the reward array = (S, A) = {env.R.shape}")
print(f"Shape of the transition array = (S, A, S) = {env.P.shape}")
print(f"Reward at (s, a) = (1, 0): {env.R[1, 0]}")
print(f"Prob[s\'=2 | s=1, a=0]: {env.P[1, 0, 2]}")
print(f"Number of states and actions: {env.Ns}, {env.Na}")

# The states in the griworld correspond to (row, col) coordinates.
# The environment provides a mapping between (row, col) and the index of
# each state:
print(f"Index of state (1, 0): {env.coord2index[(1, 0)]}")
print(f"Coordinates of state 5: {env.index2coord[5]}")

# Finding the optimal policy

**Complete the code below in order to run value iteration in a tabular MDP with transition P and rewards R.** 

In [ ]:
def value_iteration(P, R, gamma=0.95, tol=1e-3):
    """
    Args:
        P: np.array
            transition matrix (NsxNaxNs)
        R: np.array
            reward matrix (NsxNa)
        gamma: float
            discount factor
        tol: float
            precision of the solution
    Return:
        Q: final Q-function (at iteration n)
        greedy_policy: greedy policy wrt Qn
    """
    Ns, Na = R.shape
    Q = np.zeros((Ns, Na)) # current Q function 
    V = np.zeros(Ns) # current value function
    
    while True:
        TQ = np.zeros((Ns, Na))
        ### TO BE COMPLETED 
        V = TQ.max(axis=1)

        if np.abs(TQ-Q).max() < tol:
            break
        Q = TQ
    
    greedy_policy = np.argmax(Q, axis=1)
    # ====================================================
    return Q, greedy_policy

### Testing your code

In [ ]:
# Parameters
tol = 1e-5
gamma = 0.95

# Environment
env = get_env()

# run value iteration to obtain Q-values
VI_Q, VI_greedypol = value_iteration(env.P, env.R, gamma=gamma, tol=tol)

# render the policy
print("[VI]Greedy policy: ")
print(VI_greedypol)
render_policy(env, VI_greedypol)

# Collecting a database of transition

To run Fitted-Q Iteration, we will need a database of transitions that convers the MDP well enough. We first compare two ways of collecting such a dataset, given in the code below. 


In [ ]:
def get_random_policy_dataset(env, n_samples):
  """Get a dataset following a random policy to collect data."""
  states = []
  actions = []
  rewards = []
  next_states = []
  
  state = env.reset()
  for _ in range(n_samples):
    action = env.action_space.sample()
    next_state, reward, is_terminal, info = env.step(action)
    states.append(state)
    actions.append(action)
    rewards.append(reward)
    next_states.append(next_state)
    # update state
    state = next_state
    if is_terminal:
      state = env.reset()

  dataset = (states, actions, rewards, next_states)
  return dataset

def get_uniform_dataset(env, n_samples):
  """Get a dataset by uniformly sampling states and actions."""
  states = []
  actions = []
  rewards = []
  next_states = []
  for _ in range(n_samples):
    state = env.observation_space.sample()
    action = env.action_space.sample()
    next_state, reward, is_terminal, info = env.sample(state, action)
    states.append(state)
    actions.append(action)
    rewards.append(reward)
    next_states.append(next_state)

  dataset = (states, actions, rewards, next_states)
  return dataset


# Collect two different datasets
num_samples = 500
env = get_env()
dataset_1 = get_random_policy_dataset(env, num_samples)
dataset_2 = get_uniform_dataset(env, num_samples)


For each of the datasets that are built:

1. Estimate the transitions and the rewards, $\hat{P}$ and $\hat{R}$.
2. Compute the optimal value function and the optimal policy with respect to the estimated MDP (defined by $\hat{P}$ and $\hat{R}$), which we denote by $\hat{\pi}$ and $\hat{V}$.
3. Numerically compare the performance of $\hat{\pi}$ and $\pi^\star$ (the true optimal policy), and the error between $\hat{V}$ and $V^*$ (the true optimal value function).

Which of the two data collection methods do you think is better? Why?

In [ ]:
def estimate_mdp(dataset, Ns, Na):
  ## TO BE COMPLETED 
  return P_hat, R_hat

P_hat_1, R_hat_1 = estimate_mdp(dataset_1, env.Ns, env.Na)
P_hat_2, R_hat_2 = estimate_mdp(dataset_2, env.Ns, env.Na)

Q_true, pi_true = value_iteration(env.P, env.R, gamma=gamma, tol=tol)
Q_hat_1, pi_hat_1= value_iteration(P_hat_1, R_hat_1, gamma=gamma, tol=tol)
Q_hat_2, pi_hat_2 = value_iteration(P_hat_2, R_hat_2, gamma=gamma, tol=tol)

print(f"Error with random policy dataset: {np.abs(Q_hat_1-Q_true).max()}")
print(f"Error with uniform dataset: {np.abs(Q_hat_2-Q_true).max()}")

render_policy(env, pi_hat_1, horizon=100)
render_policy(env, pi_hat_2, horizon=100)

# Larger gridworlds: Fitted-Q

We will now consider a larger Gridworld, in which standard reinforcement learning algorithm like Q-Learning will typically take a very long time to converge.

In [ ]:
def get_large_gridworld():
  """Creates an instance of a grid-world MDP with more states."""
  walls = [(ii, 10) for ii in range(15) if (ii != 7 and ii != 8)]
  env = GridWorld(
      nrows=15,
      ncols=15,
      reward_at = {(14, 14):1.0},
      walls=tuple(walls),
      success_probability=0.9,
      terminal_states=((14, 14),)
  )
  return env

env = get_large_gridworld()
render_policy(env)

Given a datset $(s_i, a_i, r_i, s_i')$ of (states, actions, rewards, next states), the Fitted Q-Iteration (FQI) algorithm proceeds as follows:


* We start from a $Q$ function $Q_0 \in \mathcal{F}$, where $\mathcal{F}$ is a function space;
* At every iteration $k$, we compute $Q_{k+1}$ as:

$$
Q_{k+1}\in\arg\min_{f\in\mathcal{F}} \frac{1}{2}\sum_{i=1}^N
\left(
  f(s_i, a_i) - y_i^k
\right)^2 + \lambda \Omega(f)
$$
where $y_i^k = r_i + \gamma \max_{a'}Q_k(s_i', a')$, $\Omega(f)$ is an (optional) regularization term and $\lambda > 0$ is the regularization coefficient.


We will implement FQI with *linear* function approximation and assume a state feature map $\phi : S \rightarrow \mathbb{R}^d$. We define $\mathcal{F}$ to be the parametric family of $Q$ functions $Q_\theta(s,a) = \phi(s)^T\theta_a$ for $\theta_a\in\mathbb{R}^d$. 


We propose below to implement the feature map as a class whose method "map" applied to a state $s$ return $\phi(s)$. 

How are the features constructed? You may propose other features map(s) later to see their impact on the algorithm. 

**Complete the method "Q_table" which takes as an input a vector $\theta \in \mathbb{R}^{d\times A}$ parameterizing the Q-function and outputs the corresponding Q table in $\mathbb{R}^{S \times A}$.**

In [ ]:
class FeatureMap:
    def __init__(self, env, dim=15, sigma=0.25):
        self.index2coord = env.index2coord
        self.n_states = env.Ns
        self.n_actions = env.Na
        self.dim = dim
        self.sigma = sigma

        n_rows = env.nrows
        n_cols = env.ncols

        # build uniform discretization 
        m=int(np.floor(np.sqrt(self.dim)))
        # X and Y coordinates of the centers
        XS, YS = np.meshgrid(np.linspace(0.01,0.99, m),np.linspace(0.01,0.99, m))
        XS=XS.flatten()
        YS=YS.flatten()
    
        tot = m*m
        while (tot < self.dim):
            XS = np.hstack((XS,np.random.rand()))
            YS = np.hstack((YS,np.random.rand()))
            tot +=1
        #print(np.size(XS))
           
        # build feature matrix 
        features = np.zeros((self.dim,self.n_states))
    
        for d in range(self.dim):
            xd = XS[d]
            yd = YS[d]
            for ii in range(self.n_states):
                row_ii, col_ii = self.index2coord[ii]
                x_ii = row_ii / n_rows
                y_ii = col_ii / n_cols
                dist = np.sqrt((xd - x_ii) ** 2.0 + (yd - y_ii) ** 2.0)
                features[d, ii] = np.exp(-(dist / sigma) ** 2.0)
    
        self.feats = features
    
    def map(self, observation):
        feat = self.feats[:, observation].copy()
        return feat

    def Q_table(self,theta):
        Q = np.zeros((self.n_states, self.n_actions))
        ## TO BE COMPLETED 
        return Q
    


This code is to get a sense of what is inside a feature map 

In [ ]:
feat_map = FeatureMap(env)

# The features have dimension (feature_dim).
feature_example = feat_map.map(1) # feature representation of s=1
print("feature vector of state 1:")
print(feature_example)

# Initial vector theta representing the Q function
theta = np.random.rand(feat_map.dim, env.action_space.n)
print("the shape of theta is")
print(theta.shape)

print("values of Q(s=1,.):")
Q=feat_map.Q_table(theta)
print(Q[1,:])

**Implement Linear Fitted Q Iteration, taking as an input a feature map a number of iterations and the number of samples used in the training database (shared accross iterations).** 

For the regression problem, we propose to use $\frac{1}{2}\sum_a ||\theta_a||_2^2$ as the regularization term. Assuming that we have a given dataset of $N$ tuples of the form $(s_i, a_i, r_i, s_i')$ and we are at the $k$-th iteration. Let $\theta_k \in\mathbb{R}^{d \times A}$ be our current parameter. You will need to derive the closed-form update to find $\theta_{k+1}$.


In [ ]:
def linear_fqi(env, feat_map, num_iterations,n_samples = 10000,lambd=0.1,gamma=0.95):
    ## TO BE COMPLETED
    return theta


Running the algorithm and visualizing the value and policies that are learnt.

In [ ]:
env = get_large_gridworld()
feat_map = FeatureMap(env, dim=10, sigma=0.2)

# FQI
theta = linear_fqi()

# Compute and run greedy policy
Q_fqi = feat_map.Q_table(theta)
V_fqi = Q_fqi.max(axis=1)
policy = Q_fqi.argmax(axis=1)

render_policy(env, policy, horizon=100)
img = env.get_layout_img(V_fqi)
plt.imshow(img)

Q_true, pi_true = value_iteration(env.P, env.R, gamma=gamma, tol=tol)
V_true = Q_true.max(axis=1)

print("Error on the value:",np.abs(V_fqi-V_true).max())
